In [1]:
import json
from openai import OpenAI
import pandas as pd
from IPython.display import Image, display

In [2]:
client = OpenAI()

In [3]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [24]:
file_path = r"C:\Users\ASUS\Downloads\beforelabeled.xlsx"
df = pd.read_excel(file_path)

In [25]:
df.head()

,comment
0,did he come around
1,why did she smile
2,when i found my lost toy after 12 years later
3,hahahaha
4,she is an exceptional judge i love her respec...


In [4]:
#batch0 info
"""{
  "object": "file",
  "id": "file-Vfj8oYoSjok51WSZoqfgYH",
  "purpose": "batch",
  "filename": "file",
  "bytes": 45062909,
  "created_at": 1734048894,
  "status": "processed",
  "status_details": null
}"""

'{\n  "object": "file",\n  "id": "file-Vfj8oYoSjok51WSZoqfgYH",\n  "purpose": "batch",\n  "filename": "file",\n  "bytes": 45062909,\n  "created_at": 1734048894,\n  "status": "processed",\n  "status_details": null\n}'

In [8]:
batch_job = client.batches.create(
  input_file_id="file-63B3ZK9TuePR8UcKgitWK1",
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [14]:
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_675b8265fd5c8190b9daeff8a0a489a4', completion_window='24h', created_at=1734050406, endpoint='/v1/chat/completions', input_file_id='file-63B3ZK9TuePR8UcKgitWK1', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1734051213, error_file_id=None, errors=None, expired_at=None, expires_at=1734136806, failed_at=None, finalizing_at=1734051205, in_progress_at=1734050406, metadata=None, output_file_id='file-Qb37kyz876W5qL7LTZKLrm', request_counts=BatchRequestCounts(completed=100, failed=0, total=100))


In [15]:
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

In [20]:
result_file_name = r"C:\Users\ASUS\Downloads\data\batch_job_results_movies.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)

In [21]:
# Loading data from saved file
results = []
with open(result_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [26]:
#test
for res in results[:100]:
    task_id = res['custom_id']
    index = task_id.split('-')[-1]
    result = res['response']['body']['choices'][0]['message']['content']
    comment = df.iloc[int(index)]
    comment = comment['comment']
    print(f"comment:{comment} \n sentiment: {result}")
    print("\n\n----------------------------\n\n")

comment:did he come around  
 sentiment: {
    "sentiment": "neutral"
}


----------------------------


comment:why did she smile   
 sentiment: {
    "sentiment": "neutral"
}


----------------------------


comment:when i found my lost toy after 12 years later 
 sentiment: {
    "sentiment": "positive"
}


----------------------------


comment:hahahaha 
 sentiment: {
    "sentiment": "positive"
}


----------------------------


comment:she is an exceptional judge  i love her respect to other people  she has a good heart reminds me of judge frank caprio who i simply adore and love they are a perfect example to other judges  
 sentiment: {
    "sentiment": "positive"
}


----------------------------


comment:the coolest kid in middle school        
 sentiment: {
    "sentiment": "positive"
}


----------------------------


comment:maybe he needed that reminder   that he was the nicest kid in the middle school  
 sentiment: {
    "sentiment": "positive"
}


------------------------

In [ ]:
df['sentiment'] = np.nan

# Process the results and update the DataFrame
for res in results:
    task_id = res['custom_id']
    index = int(task_id.split('-')[-1])  # Extract the index from the task_id
    result = res['response']['body']['choices'][0]['message']['content']
    sentiment = eval(result)['sentiment']  # Convert result to dictionary and extract sentiment
    
    # Update the sentiment column at the corresponding index
    df.at[index, 'sentiment'] = sentiment

# Display a sample of the updated DataFrame
print(df.head(100))

                                              comment sentiment
0                                 did he come around    neutral
1                                 why did she smile     neutral
2       when i found my lost toy after 12 years later  positive
3                                            hahahaha  positive
4   she is an exceptional judge  i love her respec...  positive
..                                                ...       ...
95  bbc wont post 2024they cant move over affirmat...  negative
96   9 7  8 40 pm  saiful  dude  yes  it  39 s ava...   neutral
97  this fkhead got caught for a string of robberi...  negative
98                   now this video feels different     neutral
99                         how old we become my dear    neutral

[100 rows x 2 columns]


C:\Users\ASUS\AppData\Local\Temp\ipykernel_24332\2767953446.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'neutral' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'sentiment'] = sentiment


In [30]:
output_file_path = r"C:\Users\ASUS\Downloads\afterlabeled.xlsx"
df.to_excel(output_file_path, index=False)